# Using PowerSystems to calculate network matrices

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSystems.jl supports the calculation of several different matrix representations of
power system networks. This example demonstrates how to use PowerSystems.jl to calculate:
 - Y bus
 - Power transfer distribution factor (PTDF)
 - Line outage distribution  factor (LODF)

### Dependencies
Let's use a dataset from the [tabular data parsing example](../../notebook/2_PowerSystems_examples/parse_matpower.ipynb)

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "2_PowerSystems_examples", "parse_matpower.jl"))

Project SIIPExamples v0.0.2
Status `~/Documents/repos/SIIPExamples.jl/Project.toml`
  [336ed68f] CSV v0.8.0
  [9961bab8] Cbc v0.7.1
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.22.0
  [2cd47ed4] InfrastructureSystems v1.0.4
  [b6b21f68] Ipopt v0.6.3
  [2535ab7d] JSON2 v0.3.2
  [98b081ad] Literate v2.7.0
  [f0f68f2c] PlotlyJS v0.14.0
  [5f7eddb3] PowerGraphics v0.6.1
  [e690365d] PowerSimulations v0.8.0
  [bcd98974] PowerSystems v1.0.2
  [9e3dc215] TimeSeries v0.19.1
  [f269a46b] TimeZones v1.5.0
  [0f1e0344] WebIO v0.8.15
  [ade2ca70] Dates
  [56ddb016] Logging
  [44cfe95a] Pkg
  [9a3f8284] Random
  [10745b16] Statistics
  [8dfed614] Test
┌ Info: extending matpower format with data: areas 1x3
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/pm_io/matpower.jl:332
┌ Info: extending matpower format with data: gen_name 7x4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/pm_io/matpower.jl:332
┌ Info: extending matp

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,6
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
4,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
5,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
7,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
8,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
9,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5


### Ybus

In [2]:
ybus = Ybus(sys)

┌ Info: The System has no islands
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/utils/network_calculations/ybus_calculations.jl:137


PowerNetworkMatrix
:
  22.2507-222.484im  -3.52348+35.2348im  …  -15.4703+154.703im
 -3.52348+35.2348im   12.6911-126.898im          0.0+0.0im
      0.0+0.0im      -9.16758+91.6758im          0.0+0.0im
  -3.2569+32.569im        0.0+0.0im         -3.33367+33.3367im
 -15.4703+154.703im       0.0+0.0im           18.804-188.021im

### PTDF

In [3]:
ptdf = PTDF(sys)

PowerNetworkMatrix
:
  0.35088   0.0  -0.53404     0.169943   0.100401
  0.41667   0.0   0.342801    0.201807   0.119226
  0.23245   0.0   0.19124    -0.37175   -0.219627
 -0.35088   0.0  -0.46596    -0.169943  -0.100401
  0.23245   0.0   0.19124     0.62825   -0.219627
  0.116225  0.0   0.0956199   0.314125   0.390186
  0.116225  0.0   0.0956199   0.314125   0.390186

### LODF

In [4]:
lodf = LODF(sys)

PowerNetworkMatrix
:
 -1.0       0.601512   0.457143   1.0       0.457143   0.164642   0.164642
  0.6419   -1.0        0.542857  -0.6419    0.542857   0.195512   0.195512
  0.3581    0.398488  -1.0       -0.3581   -1.0       -0.360154  -0.360154
  1.0      -0.601512  -0.457143  -1.0      -0.457143  -0.164642  -0.164642
  0.3581    0.398488  -1.0       -0.3581   -1.0       -0.360154  -0.360154
  0.17905   0.199244  -0.5       -0.17905  -0.5       -1.0        0.639846
  0.17905   0.199244  -0.5       -0.17905  -0.5        0.639846  -1.0

### Indexing
Note that the axes of these matrices that correspond to buses are indexed by bus number
(::Int64) while the branch axes are indexed by branch name (::String). You can access
specific elements of the matrices as follows:

In [5]:
ptdf["bus3-bus4-i_6", 3]

0.39018648100730935

Additionally, PowerSystems provides accessors to the network matrices that take `Componets`
as arguments so that you can pass references to the components themselves rather than the
name or number. For example:

In [6]:
buses = collect(get_components(Bus, sys))
ybus[buses[1], buses[2]]

-3.2569046378322044 + 32.56904637832204im

If you would instead like to index by bus name, something like the following should work:

In [7]:
busname2num = get_components(Bus, sys) |> (c -> Dict(zip(get_name.(c), get_number.(c))))
ptdf["bus3-bus4-i_6", busname2num["bus3"]]

0.39018648100730935

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*